In [1]:
import subprocess
import os
import torch


In [9]:
DATA_PATH = "/home/jack/Raw_Tertiary_Data/"

def get_data_splits(split_type="das"):
    """
    Returns train, val, test data splits as lists.
    """
    data_list = list(torch.load(os.path.join(DATA_PATH, "processed.pt")).values())
    
    def index_list_by_indices(lst, indices):
        # return [lst[index] if 0 <= index < len(lst) else None for index in indices]
        return [lst[index] for index in indices]
    
    # Pre-compute using notebooks/split_{split_type}.ipynb
    train_idx_list, val_idx_list, test_idx_list = torch.load(
        os.path.join(DATA_PATH, f"{split_type}_split.pt")) 
    # train_list = index_list_by_indices(data_list, train_idx_list)
    # val_list = index_list_by_indices(data_list, val_idx_list)
    test_list = index_list_by_indices(data_list, test_idx_list)

    return test_list


In [10]:
test = get_data_splits()

/tmp/ipykernel_1216709/1672944786.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_list = list(torch.load(os.path.join(DATA_PATH, "processed.pt")).values())
/tmp/ipy

IndexError: list index out of range

In [15]:
train_id_list = [x["id_list"] for x in train]

In [19]:
valid_id_list = [x["id_list"] for x in val]

In [20]:
test_id_list = [x["id_list"] for x in test]

In [16]:
train_id_list[0]

['6VA1_1_A-B', '6VA2_1_A-B', '6VA4_1_A-B', '6VA3_1_A-B']

In [17]:
train = ""

In [21]:
valid = ""
test = ""

In [55]:
with open("train_id_list.txt", "w") as file:
    for i, train_ids in enumerate(train_id_list):
        for x in range(len(train_ids)):
            if x+1 != len(train_ids):
                file.write(train_ids[x]+",")
            else:
                file.write(train_ids[x]+"\n")

In [56]:
with open("valid_id_list.txt", "w") as file:
    for i, valid_ids in enumerate(valid_id_list):
        for x in range(len(valid_ids)):
            if x+1 != len(valid_ids):
                file.write(valid_ids[x]+",")
            else:
                file.write(valid_ids[x]+"\n")

In [57]:
with open("test_id_list.txt", "w") as file:
    for i, test_ids in enumerate(test_id_list):
        for x in range(len(test_ids)):
            if x+1 != len(test_ids):
                file.write(test_ids[x]+",")
            else:
                file.write(test_ids[x]+"\n")

In [58]:
test_ids = []
ids = []
with open("valid_id_list.txt", "r") as file:
    for line in file.readlines():
        line = line.strip()
        test_ids.append(line.split(","))

In [59]:
print(len(test_ids))

100


In [60]:
test_ids[0]

['7F8Z_1_A-B',
 '7EIA_1_G-H',
 '7EIA_1_C-D',
 '7F8Z_1_C-D',
 '7F8Z_1_G-H',
 '7F8Z_1_E-F',
 '7EIA_1_A-B',
 '7F8Z_1_I-J',
 '7EIA_1_E-F']

In [3]:
import json

pdb_files = []
structures = []
sequences = []
with open("CASP15.txt", "r") as file:
    for line in file.readlines():
        line = json.loads(line)
        pdb_files.append(line["pdb_file"])
        structures.append(line["struct"])
        # sequences.append(line["seq"])
print(pdb_files[0])
print(structures[0])
# print(sequences[0])

7zj4.pdb
((((((((((((((((((((((((((((((....)))))))))(((((((..[[[[[[.)))))))(((((((((..[[[[[[.)))))))((((((((((((((((((((((((((((((..]]]]]].))))))((((((()]]]]]].))))))(((((((((((((....(....((((....)))..)..........).)))))))))))))))((((((((((((...(..)(((((.(.(((((((....)))))).)).)))))))))))))))))(((((((((....)))))))))))))))))))))))))))))))(((((((((....))))))))))))))))))))))))))))))))...


In [4]:
def validate_rna_structure(structure):
    stack = []
    valid_pairs = {')': '(', ']': '[', '{': '}', '<': '>', 'A': 'a'}  # Closing brackets must match opening brackets
    try:
        for char in structure:
            if char in "([{<A":  # Push opening brackets onto the stack
                stack.append(char)
            elif char in ")]}>a":  # Check closing brackets
                if not stack or stack[-1] != valid_pairs[char]:
                    return False, "Unmatched closing bracket"  # Unmatched closing bracket
                stack.pop()
            elif char != '.':  # Invalid character (not ., (, ), [, or ])
                return False, "Invalid character"
        
        return len(stack) == 0, ""
    except:
        return False, "NoneType"

In [5]:
new_structures = []
new_sequences = []
new_pdb_files = []
for j, _ in enumerate(sequences):
    if structures[j] == None:
        continue
    final_struct = ""
    final_seq = ""
    for i, char in enumerate(sequences[j]):
        # print(i)
        if char in ["A", "G", "C", "U", "T"]:
            try:
                if char == "T":
                    char = "U"
                final_struct += structures[j][i]
                final_seq += char
            except:
                print(len(structures[j][i]))
                print(len(sequences[j][i]))
                break
    new_structures.append(final_struct)
    new_sequences.append(final_seq)
    new_pdb_files.append(pdb_files[j])
    

In [6]:
for structure, pdb_file in zip(structures, pdb_files):
    valid_or_not, reason = validate_rna_structure(structure)
    if valid_or_not:
        print(f"{pdb_file} is valid")
    else:
        print(f"{pdb_file} is not valid")
        print(reason)

7zj4.pdb is not valid
Unmatched closing bracket
8s95.pdb is not valid
Unmatched closing bracket
8uys.pdb is not valid
NoneType
7yr6.pdb is valid
7qr4.pdb is not valid
Unmatched closing bracket
7qr3.pdb is not valid
Unmatched closing bracket
7ptk.pdb is not valid
Unmatched closing bracket
7ptl.pdb is not valid
Unmatched closing bracket
8fza.pdb is not valid
Unmatched closing bracket
8uyg.pdb is not valid
NoneType
7yr7.pdb is valid
8btz.pdb is not valid
Unmatched closing bracket
8uyj.pdb is not valid
NoneType
8uye.pdb is not valid
NoneType


In [12]:
with open("rna_puzzles_sec_structs.txt", "w") as file:
    for struct, seq, pdb_file in zip(structures, sequences, pdb_files):
        valid_or_not, _ = validate_rna_structure(struct)
        if valid_or_not:
            temp = {"pdb_file": pdb_file, "struct": struct}
            file.write(json.dumps(temp)+"\n")

In [10]:
with open("rna_puzzles_sec_structs_only.txt", "w") as file:
    for struct in structures:
        if struct is not None:
            file.write(struct+"\n")